In [1]:
import math
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from sklearn.utils import shuffle
import nltk
import gensim
from gensim.models import Word2Vec
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

2023-07-07 14:34:50.473451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 14:34:51.259959: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-07 14:34:52.858224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-07 14:34:52.858349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

### Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target. 
The vocabulary is the the file `vocab.txt` (generated in the other script).

In [2]:
# Word string -> ID mapping
dictionary = dict()

with open('data/vocab.30K.txt', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', len(dictionary), end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('.', 3), ('the', 4), (',', 5), ('a', 6), ('?', 7), ('to', 8), ('you', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '.'), (4, 'the'), (5, ','), (6, 'a'), (7, '?'), (8, 'to'), (9, 'you')]
Vocabulary size:  30000


### Loading the Dataset
Here we load the data from the `dataset.csv` file (generated in the other script)

In [3]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [4]:
def lowerDataset(data):
    return data.str.lower() 
    
def cleanDataset(data):
    return data.str.replace('/r/','')                  \
                .str.replace(')','', regex=False)      \
                .str.replace('(','', regex=False)      \
                .str.replace(']','', regex=False)      \
                .str.replace('[','', regex=False)      \
                .str.replace('!','')                   \
                .str.replace('"','')                   \
    
def paddDataset(data):
    return data.str.replace(',', ' ,')                 \
                .str.replace('.',' . ', regex=False)    \
                .str.replace('?',' ?', regex=False)    \
                .str.replace('\n',' ')


In [5]:
wt = nltk.tokenize.WhitespaceTokenizer()
for column in dataset.columns:    
    dataset[column] = lowerDataset(dataset[column]) 
    dataset[column] = cleanDataset(dataset[column])
    dataset[column] = paddDataset(dataset[column])                                    
    dataset[column] = dataset[column].apply(wt.tokenize)
dataset = shuffle(dataset)

In [6]:
dataset.head()

,question,answer
2087,"[people, who, don't, want, children, ,, why, ?]","[it, only, took, me, about, 3, minutes, of, ba..."
405802,"[getting, rid, of, my, porn, addiction]","[i'm, not, going, to, tell, you, that, you, sh..."
631117,"[waiters/waitresses, of, reddit, ,, have, you,...","[blind, date, ., guy, made, a, joke, in, front..."
753087,"[how, would, you, explain, what, a, casserole,...","[take, all, the, leftovers, in, your, fridge, ..."
54697,"[you, are, stuck, in, an, elevator, with, the,...","[omg, please, dont, lock, me, in, an, elevator..."


### Data analysis
Mean sentence length and standard deviation of sentence length

In [15]:
print('Central tendency, dispersion and shape of questions’s distribution')
print(dataset['question'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of questions’s distribution
count    1149819.000000
mean          17.113906
std            9.139078
min            1.000000
25%           11.000000
50%           15.000000
75%           21.000000
max           82.000000
Name: question, dtype: object


In [16]:
print('Central tendency, dispersion and shape of answers’s distribution')
print(dataset['answer'].str.len().describe().apply(lambda x: format(x, 'f')))

Central tendency, dispersion and shape of answers’s distribution
count    1149819.000000
mean          54.452828
std          844.371854
min            0.000000
25%           10.000000
50%           22.000000
75%           53.000000
max       563680.000000
Name: answer, dtype: object


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [7]:
max_sent_length = {'question' : 30, 'answer': 60}

def padding_sent(source):
    padded = []
    for tokens in dataset[source]: 
        # adding the start token
        tokens.insert(0, '<s>')  

        if len(tokens) >= max_sent_length[source]:
            tokens = tokens[:(max_sent_length[source] - 1)]
            tokens.append('</s>')

        if len(tokens) < max_sent_length[source]:
            tokens.extend(['</s>' for _ in range(max_sent_length[source] - len(tokens))])  

        padded.append(tokens)
    return padded

In [8]:
questions = padding_sent('question')
answers = padding_sent('answer')

### Create the reverse dataset
The reverse dataset are going to be used to retrieve the decoder output to text

In [9]:
def create_reverse_dataset(source):
    reverse_tokens = []
    reverse_dataset = []
    for tokens in source: 
        for token in tokens: 
            if token not in dictionary.keys():
                reverse_tokens.append(dictionary['<unk>'])
            else:
                reverse_tokens.append(dictionary[token])
        reverse_dataset.append(reverse_tokens)
        reverse_tokens = []
    return reverse_dataset

inputs_indexes =  np.array(create_reverse_dataset(questions), dtype=np.int32)
outputs_indexes =  np.array(create_reverse_dataset(answers), dtype=np.int32)

### Word Embedding
Use the Word2Vec to embed the input to a hight demention vector, that will keep the word relationships 

In [11]:
"""
Learns all the Word2Vec relationships fo the questions, answers and unkown words
This code needs to be run just once
"""

model = Word2Vec(questions + answers + [['<unk>']], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [10]:
model = Word2Vec.load("word2vec.model")

### Formating the inputs and outputs

The dataset original format for inputs is 30x100 and for the outputs is 60x30000 for 1.149 million records, making the memory usage impracticable.

The inputs and outputs are goint to be refactor to 15x100 for the inputs and 30x30000 for the outputs to a total of 2.299 million records.

In [282]:
"""

input_window_size = 15
output_window_size = 30

"""

#Factor to reshape the dataset
reshape_factor = 2 

input_window_size = int(max_sent_length['question'] / reshape_factor)
output_window_size = int(max_sent_length['answer'] / reshape_factor)

def array_numpy(array):
    return np.array(array, dtype=np.float32)

def get_batch_inputs(batch, batch_size = 10):
    train_inputs = list()
    
    for input_index in inputs_indexes[batch:batch + batch_size]:
        train_input = list()   
        
        for index in input_index:
            # Formates the input to the word2vec encoded format
            train_input.append(model.wv[reverse_dictionary[index]])
            
        train_inputs.append(array_numpy(train_input[:input_window_size]))
        train_inputs.append(array_numpy(train_input[input_window_size:]))
    return array_numpy(train_inputs)

def get_batch_outputs(batch, batch_size = 10):
    train_outputs = list()
    train_targets = list()
    
    for output_index in outputs_indexes[batch:batch + batch_size]:
        train_output = list()
        train_target = list()
                
        for timestep, index in enumerate(output_index):
            # Formates the output to the one-hot-encode format
            output_encoded = np.zeros(len(dictionary), dtype=np.float32)
            output_encoded[index] = 1
            train_output.append(output_encoded)
            
            # Formates the target to the one-hot-encode format
            # Setted as index - 1 because it ignores the first <s>
            if timestep > 0:
                target_encoded = np.zeros(len(dictionary), dtype=np.float32)
                target_encoded[output_index[timestep]] = 1.0
                train_target.append(target_encoded)
        
        train_outputs.append(array_numpy(train_output[:output_window_size]))
        train_outputs.append(array_numpy(train_output[output_window_size:]))
        
        #Add a </s> in the end of the target so len(output) == len(taget) 
        target_encoded = np.zeros(len(dictionary), dtype=np.float32)
        target_encoded[output_index[-1]] = 1.0
        train_target.append(target_encoded)
                
        train_targets.append(array_numpy(train_target[:output_window_size]))
        train_targets.append(array_numpy(train_target[output_window_size:]))
        
    return array_numpy(train_outputs), array_numpy(train_targets)

In [283]:
input_example = get_batch_inputs(0, 1)
output_example, target_example = get_batch_outputs(0, 1)

In [289]:
# Input shape = 100
input_shape = input_example.shape[2]

# Output shape = 30000
output_shape = output_example.shape[2]

#Dimensionality
dimensionality = 256

#The batch size and number of epochs
batch_size = 10
epochs = 600

#Encoder
encoder_inputs = Input(shape=(None, input_shape))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, output_shape))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_shape, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')


2023-07-07 17:14:16.472317: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 117.19MiB (rounded to 122880000)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-07 17:14:16.472388: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-07-07 17:14:16.472427: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 43, Chunks in use: 43. 10.8KiB allocated for chunks. 10.8KiB in use in bin. 240B client-requested in use in bin.
2023-07-07 17:14:16.472454: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-07-07 17:14:16.472480: I tensorflow/core/common

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

In [ ]:
# num_records = len(dataset) * reshape_factor

# data_batch_size = 1000

# batch_records = math.ceil(num_records / data_batch_size)

# supported_batch_records = num_records / data_batch_size

# Run for the batches 
for batch_record in range(batch_records + 1):
    
    # Validate for the limit size of the dataset
    # Limit the last step not to exceed the data size
    if batch_record > (num_records / data_batch_size):
        input_batch_size = math.floor((supported_batch_records % 1) * data_batch_size)
    
    # Get the inputs
    inputs_data = get_batch_inputs(batch_record, data_batch_size)
    
    # Get the outputs
    outputs_data, target_data = get_batch_outputs(batch_record, data_batch_size)
    
    #Training
    #training_model.fit([inputs_data, outputs_data], outputs_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
    #training_model.save('training_model.h5')



In [290]:
inputs_data = get_batch_inputs(0,100)
    
outputs_data, target_data = get_batch_outputs(0,100)

print(inputs_data.shape)
print(outputs_data.shape)
print(target_data.shape)


(200, 15, 100)
(200, 30, 30000)
(200, 30, 30000)


In [292]:
training_model.fit([inputs_data, outputs_data], target_data, batch_size = 10, epochs = 10, validation_split = 0.2)

Epoch 1/10
16/16 [==============================] - 3s 183ms/step - loss: 3.3291 - accuracy: 0.4992 - val_loss: 4.3018 - val_accuracy: 0.5542
Epoch 2/10
16/16 [==============================] - 2s 121ms/step - loss: 3.3153 - accuracy: 0.4994 - val_loss: 4.3571 - val_accuracy: 0.5550
Epoch 3/10
16/16 [==============================] - 2s 130ms/step - loss: 3.2950 - accuracy: 0.5008 - val_loss: 4.3329 - val_accuracy: 0.5542
Epoch 4/10
16/16 [==============================] - 2s 120ms/step - loss: 3.2703 - accuracy: 0.4994 - val_loss: 4.3602 - val_accuracy: 0.5567
Epoch 5/10
16/16 [==============================] - 2s 122ms/step - loss: 3.2541 - accuracy: 0.5019 - val_loss: 4.3320 - val_accuracy: 0.5558
Epoch 6/10
16/16 [==============================] - 2s 119ms/step - loss: 3.2239 - accuracy: 0.5013 - val_loss: 4.3906 - val_accuracy: 0.5542
Epoch 7/10
16/16 [==============================] - 2s 119ms/step - loss: 3.1901 - accuracy: 0.5013 - val_loss: 4.3449 - val_accuracy: 0.5542
Epoch 